Reset the Kernel before Running for the 2nd time.

In [2]:
import tensorflow as tf
import numpy as np
import codecs
import collections
from six.moves import cPickle
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq

# TESTING

#### ARGUMENTS

In [3]:
# Specify choosen directory from where to load the pre-trained model
saveDirectory = "data/model"
charsVocabFile = saveDirectory + "/chars_vocab.pkl"

In [4]:
# Open saved pickle files
with open(charsVocabFile, 'rb') as f:
    chars, vocabMapper = cPickle.load(f)

#### ARGUMENTS

In [5]:
# Same configuration (parameters) in training part
layersCount = 2
RNNSize = 128
batchSize = 1
sequenceLength = 1
vocabSize = len(chars)

### MODEL CREATION

In [6]:
cellFunction = rnn.BasicLSTMCell
# Crete a list contains RNN cells for each layer
cells = list()
for _ in range(layersCount):
    cell = cellFunction(RNNSize)
    cells.append(cell)
# Crate RNN cells according to created list
cell = rnn.MultiRNNCell(cells, state_is_tuple=True)
# Set placeholder for input and target
inputData = tf.placeholder(tf.int32, [batchSize, sequenceLength])
targets = tf.placeholder(tf.int32, [batchSize, sequenceLength])
# Set initial state to all zeros
stateInitial = cell.zero_state(batchSize, tf.float32)
# Set rnnlm scope according to given arguments
with tf.variable_scope('rnnlm'):
    weights = tf.get_variable("weights", [RNNSize, vocabSize])
    biases = tf.get_variable("biases", [vocabSize])
# Create matrix for embedding and inputs 
embedding = tf.get_variable("embedding", [vocabSize, RNNSize])
inputs = tf.nn.embedding_lookup(embedding, inputData)
inputs = tf.split(inputs, sequenceLength, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
# Specify the loop function that will be used for determinin outputs
def loop(previous, _):
    previous = tf.matmul(previous, weights) + biases
    previousSymbol = tf.stop_gradient(tf.argmax(previous, 1))
    return tf.nn.embedding_lookup(embedding, previousSymbol)
outputs, lastState = legacy_seq2seq.rnn_decoder(inputs, stateInitial, cell, loop_function=loop, scope='rnnlm')
output = tf.reshape(tf.concat(outputs, 1), [-1, RNNSize])
logits = tf.matmul(output, weights) + biases
probabilities = tf.nn.softmax(logits)
# Define a cost function
loss = legacy_seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(targets, [-1])],
        [tf.ones([batchSize * sequenceLength])])
cost = tf.reduce_sum(loss) / batchSize / sequenceLength
with tf.name_scope('cost'):
    cost = tf.reduce_sum(loss) / batchSize / sequenceLength
finalState = lastState
lr = tf.Variable(0.0, trainable=False)
trainables = tf.trainable_variables()
gradients, _ = tf.clip_by_global_norm(tf.gradients(cost, trainables), 5.0)
# Define an optimizer whihch is AdamOptimizer in this case since it performs well
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(lr)
# Define the train operation
trainOperation = optimizer.apply_gradients(zip(gradients, trainables))

#### ARGUMENTS

In [7]:
# Define certain arguments before running the session
sampleCharCount = 1000
primerText = u' '

## SESSION

In [8]:
# Define a function for picking the most likely state
# It is used instead of just argmnax()
def weightedPick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))

# Run the session to create the text according to pre-trained model
# And save the generated text
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    # Load trained model
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(saveDirectory)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        state = sess.run(cell.zero_state(1, tf.float32))
        for char in primerText[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocabMapper[char]
            feed = {inputData: x, stateInitial: state}
            [state] = sess.run([finalState], feed)

        returnText = primerText
        char = primerText[-1]
        for n in range(sampleCharCount):
            x = np.zeros((1, 1))
            x[0, 0] = vocabMapper[char]
            feed = {inputData: x, stateInitial: state}
            [probabilitiesV2, state] = sess.run([probabilities, finalState], feed)
            p = probabilitiesV2[0]
            sample = weightedPick(p)
            prediction = chars[sample]
            returnText += prediction
            char = prediction
        

    file = open("data/text/generated/current.txt","w") 
    file.write(str(returnText))
    file.close() 

INFO:tensorflow:Restoring parameters from data/model/model.ckpt-2054


### End of the text generation